# Wiener Process and Geometric Brownian Motion

## Question 1: Simulate a Single Path of a Wiener Process

Discretize \([0, T]\) into \(N\) steps of size \(\Delta t = T/N\).  
Increment distribution:

\[
W_{t_{i+1}} - W_{t_i} \sim N(0, \Delta t).
\]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
T = 1.0
N = 1000
dt = T / N

# Time grid and increments
t = np.linspace(0, T, N+1)
dW = np.random.normal(0, np.sqrt(dt), size=N)

# Construct path
W = np.empty(N+1)
W[0] = 0
W[1:] = np.cumsum(dW)

# Plot
plt.figure(figsize=(8,4))
plt.plot(t, W)
plt.title("Single Wiener Process Path")
plt.xlabel("t")
plt.ylabel("W(t)")
plt.grid(True)
plt.show()


## Question 2: Simulate 5 Paths of Geometric Brownian Motion

\[
S(t) = S(0) \exp\bigl(σW(t) + (α - \tfrac12σ^2)t\bigr).
\]


In [ ]:
# Parameters
S0 = 100
alpha = 0.05
sigma = 0.2
paths = 5

plt.figure(figsize=(8,4))
for i in range(paths):
    dW = np.random.normal(0, np.sqrt(dt), size=N)
    W = np.cumsum(dW)
    t_short = np.linspace(0, T, N)
    S = S0 * np.exp(sigma * W + (alpha - 0.5*sigma**2)*t_short)
    plt.plot(t_short, S, label=f"Path {i+1}")

plt.title("Geometric Brownian Motion Paths")
plt.xlabel("t")
plt.ylabel("S(t)")
plt.legend()
plt.grid(True)
plt.show()


## Question 3: Covariance \(E[W_s W_t] = \min(s,t)\)

**Theoretical derivation:**

1. For \(s \le t\), write
   \[
   W_t = W_s + (W_t - W_s).
   \]
   Independence of increments gives
   \[
   E[W_s(W_t - W_s)] = E[W_s]E[W_t - W_s] = 0,
   \]
   so
   \[
   E[W_s W_t] = E[W_s^2] = \operatorname{Var}(W_s) = s.
   \]
2. If \(t < s\), swap roles.

Thus \(E[W_s W_t] = \min(s,t)\).


In [ ]:
import numpy as np

# Simulation parameters
M = 20000  # number of paths
s, t = 0.3, 0.7
s_idx = int(s * N)
t_idx = int(t * N)

# Simulate M paths
dW_all = np.random.normal(0, np.sqrt(dt), size=(M, N))
W_all = np.hstack([np.zeros((M,1)), np.cumsum(dW_all, axis=1)])

empirical = np.mean(W_all[:,s_idx] * W_all[:,t_idx])
theoretical = min(s, t)

print(f"Empirical E[W_s W_t] = {empirical:.4f}, Theoretical = {theoretical:.4f}")


## Question 4: Increment Distribution and Independence

1. **Distribution**  
   By stationary increments:  
   \[
     W_t - W_s \sim N(0, t-s).
   \]
2. **Independence**  
   Increments over disjoint intervals are independent by definition.

**Empirical checks:**  
- mean & variance of \(W_t-W_s\).  
- correlation between \(W_t-W_s\) and a separate increment \(W_s-W_0\).


In [ ]:
# Empirical mean & variance
increments = W_all[:,t_idx] - W_all[:,s_idx]
print("Mean:", np.mean(increments))
print("Variance:", np.var(increments), "vs t-s =", t-s)

# Independence check (correlation)
inc1 = W_all[:,s_idx] - W_all[:,0]
inc2 = increments
print("Correlation (disjoint increments):", np.corrcoef(inc1, inc2)[0,1])


## Question 5: Martingale Property

**Theory:**  
For \(0\le s\le t\),
\[
E[W_t \mid \mathcal{F}_s] 
= E[W_s + (W_t - W_s) \mid \mathcal{F}_s]
= W_s + 0 = W_s.
\]
Thus \((W_t)\) is a martingale.

- Regress \(W_t\) on \(W_s\): slope ≈ 1, intercept ≈ 0.  
- Compute binned conditional means \(E[W_t \mid W_s]\) vs. \(W_s\).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Regression
x, y = W_all[:,s_idx], W_all[:,t_idx]
slope, intercept = np.polyfit(x, y, 1)
print("Regression slope:", slope, "intercept:", intercept)

# Binned conditional means
bins = np.linspace(-3*np.sqrt(s), 3*np.sqrt(s), 10)
digitized = np.digitize(x, bins)
bin_centers = 0.5*(bins[:-1] + bins[1:])
cond_means = [np.mean(y[digitized==i]) for i in range(1,len(bins))]

# Plot
plt.figure(figsize=(6,4))
plt.plot(bin_centers, cond_means, 'o-', label='Empirical')
plt.plot(bin_centers, bin_centers, '--', label='y=x')
plt.xlabel('W_s')
plt.ylabel('E[W_t | W_s]')
plt.legend()
plt.grid(True)
plt.show()
